<a href="https://colab.research.google.com/github/osaeed-ds/bedrock/blob/main/bedrock_astra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Similarity Astra-Bedrock Search QA Quickstart

Set up a simple Question-Answering system with LangChain and Amazon Bedrock, using Astra DB as the Vector Database.

## Prerequisites

Make sure you have a vector-capable Astra database (get one for free at [astra.datastax.com](https://astra.datastax.com)):

- You will be asked to provide the **Database ID** for your Astra DB instance (see [here](https://awesome-astra.github.io/docs/pages/astra/faq/#where-should-i-find-a-database-identifier) for details);
- Ensure you have an **Access Token** for your database with role _Database Administrator_ (see [here](https://awesome-astra.github.io/docs/pages/astra/create-token/) for details).

Likewise, you will need the credentials to your Amazon Web Services identity, with access to **Amazon Bedrock**.

## Set up your Python environment

In [1]:
%pip install --quiet \
  "cassio>=0.1.3" \
  "langchain==0.0.249" \
  "boto3==1.28.62" \
  "botocore==1.31.62" \
  "cohere==4.37" \
  "openai==1.3.7" \
  "tiktoken==0.5.2" \
  "awscli==1.29.62"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 760.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Import needed libraries

In [2]:
import json
import os
import sys
from getpass import getpass


import boto3
import cassio

from langchain.embeddings import BedrockEmbeddings
from langchain.llms import Bedrock
from langchain.vectorstores import Cassandra
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader

## Astra DB Setup

In [ ]:
ASTRA_DB_ID = input("Enter your Astra DB ID ('0123abcd-'):")
ASTRA_DB_APPLICATION_TOKEN = getpass("Enter your Astra DB Token ('AstraCS:...'):")
ASTRA_DB_KEYSPACE = input("Enter your keyspace name (optional, default keyspace used if not provided):")

In [4]:
cassio.init(
    token=ASTRA_DB_APPLICATION_TOKEN,
    database_id=ASTRA_DB_ID,
    keyspace=ASTRA_DB_KEYSPACE if ASTRA_DB_KEYSPACE else None,
)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(137154635900160) 172fa27f-656f-4584-8a7e-174b794d3cf0-westus3.db.astra.datastax.com:29042:04e14236-6e3d-41d4-93b5-9c5e1beb16de> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


## AWS Credentials Setup

_Note_: in the following cells you will be asked to explicitly provide the credentials to your AWS account. These are set as environment variables for usage by the subsequent `boto3` calls. Please refer to [boto3's documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html) on the possible ways to supply your credentials in a more production-like environment.

In particular, if you are running this notebook in **Amazon SageMaker Studio**, please note that it is sufficient to add the Bedrock policy to your SageMaker role, as outlined at [this link](https://github.com/aws-samples/amazon-bedrock-workshop#enable-aws-iam-permissions-for-bedrock), to access the Bedrock services. In that case you can skip the following three setup cells.

In [ ]:
# Input your AWS Access Key ID
os.environ["AWS_ACCESS_KEY_ID"] = getpass("Your AWS Access Key ID:")

In [ ]:
# Input your AWS Secret Access Key
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass("Your AWS Secret Access Key:")

In [ ]:
# Input your AWS Session Token
os.environ["AWS_SESSION_TOKEN"] = getpass("Your AWS Session Token:")

## Set up AWS Bedrock objects

In [8]:
bedrock_runtime = boto3.client("bedrock-runtime", "us-west-2")
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                                       client=bedrock_runtime)

## Set up the Vector Store

This command will create a suitable table in your database if it does not exist yet:

In [9]:
vector_store = Cassandra(
    embedding=bedrock_embeddings,
    table_name="shakespeare_act5",
    session=None,  # <-- meaning: use the global defaults from cassio.init()
    keyspace=None,  # <-- meaning: use the global defaults from cassio.init()
)

## Populate the database

Add lines for the text of "Romeo and Astra", Scene 5, Act 3

In [10]:
# retrieve the text of a scene from act 5 of Romeo and Astra.
# Juliet's name was changed to Astra to prevent the LLM from "cheating" when providing an answer.
! mkdir -p "texts"
! curl "https://raw.githubusercontent.com/awesome-astra/docs/main/docs/pages/aiml/aws/bedrock_resources/romeo_astra.json" \
    --output "texts/romeo_astra.json"
input_lines = json.load(open("texts/romeo_astra.json"))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75985  100 75985    0     0   224k      0 --:--:-- --:--:-- --:--:--  225k


Next, you'll populate the database with the lines from the play.
This can take a couple of minutes, please be patient.  In total there are 321 lines.


In [11]:
input_documents = []

for input_line in input_lines:
    if (input_line["ActSceneLine"] != ""):
        (act, scene, line) = input_line["ActSceneLine"].split(".")
        location = "Act {}, Scene {}, Line {}".format(act, scene, line)
        metadata = {"act": act, "scene": scene, "line": line}
    else:
        location = ""
        metadata = {}
    quote_input = "{} : {} : {}".format(location, input_line["Player"], input_line["PlayerLine"])
    input_document = Document(page_content=quote_input, metadata=metadata)
    input_documents.append(input_document)

print(f"Adding {len(input_documents)} documents ... ", end="")
vector_store.add_documents(documents=input_documents, batch_size=50)
print("Done.")

Adding 321 documents ... Done.


## Answer questions

In [33]:
prompt_template_str = """Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context

Question: {question}

Assistant:"""

prompt = PromptTemplate.from_template(prompt_template_str)

We choose to use the following LLM model (see [this page](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html#model-parameters-general) for more info):

In [11]:
model_id = "anthropic.claude-v2"

Here the question-answering function is set up, implementing the RAG pattern:

In [19]:
req_accept = "application/json"
req_content_type = "application/json"

# This, created from the vector store, will fetch the top relevant documents given a text query
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

def answer_question(question: str, verbose: bool = False) -> str:
    if verbose:
        print(f"\n[answer_question] Question: {question}")
    # Retrieval of the most relevant stored documents from the vector store:
    context_docs = retriever.get_relevant_documents(question)
    context = "\n".join(doc.page_content for doc in context_docs)
    if verbose:
        print("\n[answer_question] Context:")
        print(context)
    # Filling the prompt template with the current values
    llm_prompt_str = prompt.format(
        question=question,
        context=context,
    )
    # Invocation of the Amazon Bedrock LLM for text completion -- ultimately obtaining the answer
    llm_body = json.dumps({"prompt": llm_prompt_str, "max_tokens_to_sample": 500})
    llm_response = bedrock_runtime.invoke_model(
        body=llm_body,
        modelId=model_id,
        accept=req_accept,
        contentType=req_content_type,
    )
    llm_response_body = json.loads(llm_response["body"].read())
    answer = llm_response_body["completion"].strip()
    if verbose:
        print(f"\n[answer_question] Answer: {answer}\n")
    return answer

In [13]:
my_answer = answer_question("Who dies in the story?")
print("=" * 60)
print(my_answer)

Based on the provided context, it seems that Astra dies in the story. The lines mention Astra being found dead, warm, and newly killed, as well as characters reacting to and being affected by Astra's death.


Let's take a look at the RAG process piece-wise:

In [14]:
my_answer = answer_question("Who dies in the story?", verbose=True)
print("=" * 60)
print(my_answer)


[answer_question] Question: Who dies in the story?



[answer_question] Context:
Act 5, Scene 3, Line 184 : First Watchman : And Astra bleeding, warm, and newly dead,
Act 5, Scene 3, Line 244 : FRIAR LAURENCE : Was Tybalt's dooms-day, whose untimely death
Act 5, Scene 3, Line 300 : PRINCE : Came to this vault to die, and lie with Astra.
Act 5, Scene 3, Line 282 : BALTHASAR : I brought my master news of Astra's death,
Act 5, Scene 3, Line 206 : First Watchman : Warm and new kill'd.

[answer_question] Answer: Based on the context provided, it seems that Astra is the one who dies in the story. The lines mention Astra being found dead, warm, and newly killed, as well as characters reacting to and speaking about Astra's death.

Based on the context provided, it seems that Astra is the one who dies in the story. The lines mention Astra being found dead, warm, and newly killed, as well as characters reacting to and speaking about Astra's death.


### Interactive QA session

In [27]:
user_question = ""
while True:
    user_question = input("Enter a question (empty to quit):").strip()
    if user_question:
        print(f"Answer ==> {answer_question(user_question)}")
    else:
        print("[User, AI exeunt]")
        break

Enter a question (empty to quit):what family does astra belong to?


Answer ==> Based on the context provided, it seems that Astra belongs to the Capulet family. The references to her being Romeo's love interest indicate she is playing the role of Juliet, who is a Capulet. 

In English: Astra belongs to the Capulet family.

In Pig Latin: Astra elongsbay otay ethay Apuletcay amilyfay.


KeyboardInterrupt: ignored

### Changing the prompt and see the difference in answer
The existing prompt gives instructions not to guess at an answer and reply that the answer cannot be determined. The dataset was also rigged to replace Juliet with Astra so that it will not try to search on its own.  We are going to test with a new prompt that does the following.
1. If an answer is not known, replace Astra with Juliet in the context
2. Give the answer in both English and Pig Latin

We will ask the same question with both prompts.

In [34]:
#existing prompt that does not try to guess and is unaware of Juliet
prompt_template_str = """Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context

Question: {question}

Assistant:"""

prompt = PromptTemplate.from_template(prompt_template_str)

In [35]:
my_answer = answer_question("What family does Astra belong to?")
print("=" * 60)
print(my_answer)

Based on the context provided, I do not have enough information to determine what family Astra belongs to. The lines mention Astra along with Romeo and Tybalt, who are from the Montague and Capulet families respectively, but there is no clear indication of which family Astra herself belongs to. Without more context about her relationships and background, I cannot confidently answer the question.


In [36]:
#new prompt that is aware of Juliet and will also answer in Pig Latin
prompt_template_str = """Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, replace any instances of Astra in the Context with Juliet, then attempt to answer.  give the answer in english and in pig latin.

<context>
{context}
</context

Question: {question}

Assistant:"""

prompt = PromptTemplate.from_template(prompt_template_str)

In [38]:
my_answer = answer_question("What family does Astra belong to?")
print("=" * 60)
print(my_answer)

<context>
Act 5, Scene 3, Line 321 : PRINCE : Than this of Juliet and her Romeo.  
Act 5, Scene 3, Line 313 : MONTAGUE : As that of true and faithful Juliet.
Act 5, Scene 3, Line 86 : ROMEO : For here lies Juliet, and her beauty makes
Act 5, Scene 3, Line 300 : PRINCE : Came to this vault to die, and lie with Juliet.  
Act 5, Scene 3, Line 246 : FRIAR LAURENCE : For whom, and not for Tybalt, Juliet pined.
</context>

Answer: Astra belongs to the Montague family.

In English: Juliet belongs to the Montague family.

In Pig Latin: ulietJay elongsbay otay hetay ontagueMay amilyfay.


## Additional resources

To learn more about Amazon Bedrock, visit this page: [Introduction to Amazon Bedrock](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/introduction-to-bedrock).